In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import torchvision 

In [55]:
from torch.utils.data import Dataset 
import cv2
import matplotlib.pyplot as plt
from xml.dom import minidom
from os.path import basename
import glob

In [56]:
def unpickle(file):
   import pickle
   with open(file, 'rb') as fo:
      dict = pickle.load(fo, encoding='bytes')
   return dict

In [ ]:
training=torchvision.datasets.ImageNet(root='/kaggle/input/imagenet-object-localization-challenge/ILSVRC', train=True, download=True)

In [57]:
def generate_noise(path,t,ct):
    img=cv2.imread(path)
    img=np.array(img)
    img=img/255.0
    
    epsilon=np.random.randn(*img.shape).astype(np.float32)
    noise = np.exp(-t / ct)
    noise=np.clip(noise,0,1)
    new_img=np.sqrt(noise)*img + np.sqrt(1-noise)*epsilon
    new_img=np.clip(new_img,0,1)
    
    return noise

In [58]:
xml_folder=r'/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Annotations/CLS-LOC/train'
train_folder=r'/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/*'

In [59]:
train_folders = glob.glob('/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/*')

In [60]:
folder=(train_folders[0]+'/*')

In [61]:
folder_name=glob.glob(folder)

In [ ]:
(folder_name)

In [ ]:
for i in range(1,101):
    generate_noise(folder_name[0],0.0001,0.02,i,100)

In [52]:
class Diffusion(nn.Module):
    def __init__(self, beta1,beta2,epsilon) -> None:
        self.beta = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.time_embed = nn.Linear(1, embed_dim)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
 
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 3, kernel_size=(3,3), stride=1, padding=1)

    def forward(self,x):
        x = self.act1(self.conv1(x))
        x = self.drop1(x)
        # input 32x32x32, output 32x32x32
        x = self.act2(self.conv2(x))
        # input 32x32x32, output 32x16x16
        x = self.pool2(x)
        # input 32x16x16, output 8192
        x = self.flat(x)
        # input 8192, output 512
        x = self.act3(self.fc3(x))
        x = self.drop3(x)
        # input 512, output 10
        x = self.fc4(x)
        return x

model = Diffusion()
loss_fn = nn.MSE()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
def generate(path,t,ct):
    img=cv2.imread(path)
    img=np.array(img)
    img=img/255.0
    
    epsilon=np.random.randn(*img.shape).astype(np.float32)
    noise = np.exp(-t / ct)
    noise=np.clip(noise,0,1)
    new_img=np.sqrt(noise)*img + np.sqrt(1-noise)*epsilon
    new_img=np.clip(new_img,0,1)
    return new_img,noise

def noising(path,t,epoch):
    new_img,error=generate(path,t,epoch)
    pred_error=model.forward(new_img)
    loss = loss_fn(pred_error, error)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
def train(folder,epoch):
    for e in range(len(folder)):
        path=folder[e]
        for t in range(epoch):
            noising(path,t,epoch)
        